# Playing with libs for string operations

Python's fun, yet complex ! Using libraries is a way to ease your life around some problems.

Built-ins functionalities of python like re, collections or functools are pillars for clean, optimized code.

Do not rewrite code, find a lib that does what you want ! Then, enhance it by adding more functionnalities as needed.

In [1]:
from string import whitespace, punctuation, digits
from timeit import timeit
from collections import Counter, defaultdict
import functools
import re

# INITIALISATION
# str.join method is the fastest and memory friendly compared
# to many other formatting styles (especially on loops !)
# Ensure each unwanted character is unique with a set(...)
non_chars = "".join(set([whitespace, punctuation, digits]))

text = " ".join([
    "Python's fun, yèt complex ! Using libraries is a way to eâse yöur",
    "life arouÑd some problems. Built-ins 'fonctionalités' of python like re,",
    "collectioNs or functools are \\\n pillars for a clean, optimized code.\\\n",
    "Do not rewrite code, enhance it!",
    "Now, Dø It Yourself ;)"
])


# DEFINING BASIC FUNCTIONS
def classic_get_words(_string: str) -> list[str]:
    """
    The most commonly used method to count words in a text is
    poorly optimized. On top of being slow, it does not allow proper
    counting without further operations. Changing a thing can oftentimes
    harden the debug process and lead to unwanted results.
    """
    # strip removes end-line characters, making a one-line text
    words = _string.strip().split()
    for word_index in range(len(words)):
        # If the word is alphabetic, we do not need further operations
        if not words[word_index].isalpha():
            alpha_word = ""
            for letter in words[word_index]:
                if letter not in non_chars or letter in ('\'', '-', '_'):
                    # or letter in (...) is to capture 's and linked-_-words
                    # It is done this way because :
                    # 'not in' breaks itself on first missmatch
                    # If we were to invert the tests, process would be much slower !
                    alpha_word += letter

            words[word_index] = alpha_word
    # returns only words that are not empty.
    return [word for word in words if word]


def regex_get_words(_string: str) -> list[str]:
    """
    On top of being quite clear and easy to adapt to specific cases,
    it is also pretty fast to gather anything you want in a text.
    In regex, \w is an alias for [a-zA-Z0-9_] meaning it will match ascii_letters,
    digits and symbol '_'.
    We only want words, so we have to use [a-zA-ZÇ-ý'_-]+
    Where:
    - [a-zA-Z] is the same as string.ascii_letters
    - [Ç-ý] takes accents, cedilles, and special characters into consideration
    - ['_-] allows us to capture 's and linked-_-words
    - + means 'find as much as you can in a row'.
        if any character is not one of those: next word

    We could also use negation with non-capturing expressions like (?:(?!:[a-zA-Z])
    to perform exactly like the 'classic method'
    """
    words = re.findall(r"[A-Za-zÇ-ý'_-]+", _string)
    return words

print(classic_get_words(text), "\n")
print(regex_get_words(text), "\n")
assert classic_get_words(text) == regex_get_words(text)

# In order to have better oversight on time taken for simple functions,
# we have to run them many times 
nb_runs = 100000
print(f"first, let's compare classic approach to regex findall  on {nb_runs} runs:")
print("classic :",
      f"{timeit(functools.partial(classic_get_words, text), number=nb_runs):.3f}s")
print("regex :",
      f"{timeit(functools.partial(regex_get_words, text), number=nb_runs):.3f}s")

["Python's", 'fun', 'yèt', 'complex', 'Using', 'libraries', 'is', 'a', 'way', 'to', 'eâse', 'yöur', 'life', 'arouÑd', 'some', 'problems', 'Built-ins', "'fonctionalités'", 'of', 'python', 'like', 're', 'collectioNs', 'or', 'functools', 'are', 'pillars', 'for', 'a', 'clean', 'optimized', 'code', 'Do', 'not', 'rewrite', 'code', 'enhance', 'it', 'Now', 'Dø', 'It', 'Yourself'] 

["Python's", 'fun', 'yèt', 'complex', 'Using', 'libraries', 'is', 'a', 'way', 'to', 'eâse', 'yöur', 'life', 'arouÑd', 'some', 'problems', 'Built-ins', "'fonctionalités'", 'of', 'python', 'like', 're', 'collectioNs', 'or', 'functools', 'are', 'pillars', 'for', 'a', 'clean', 'optimized', 'code', 'Do', 'not', 'rewrite', 'code', 'enhance', 'it', 'Now', 'Dø', 'It', 'Yourself'] 

first, let's compare classic approach to regex findall  on 100000 runs:
classic : 2.026s
regex : 1.217s


As we can see, __regex__ is almost twice as fast as the 'classic' method. (other optimized solutions exists)

Now we have sorted this out, in the next tests, we will exclusively use this regex method, since it is faster...

### Next step:
    Since we only want to know how fast each methodology is, we are not going to time regex findings.

In [2]:
# DEFINFING COUNTER FUNCTIONS
def counter_count_words(words):
    counter = Counter(words)
    return counter


def defaultdict_count_words(words):
    my_counter = defaultdict(int)
    for word in words:
        my_counter[word] = my_counter.get(word, 0) + 1
    return my_counter


def dict_count_words(words):
    my_counter = {}
    for word in words:
        my_counter[word] = my_counter.get(word, 0) + 1
    return my_counter


# not timed
words = regex_get_words(text)
print()
print("then, if we want to count words occurrences in a text," 
      f"using Counter or dicts  on {nb_runs} runs:")
print("default_dict :", 
      f"{timeit(functools.partial(defaultdict_count_words, words), number=nb_runs):.3f}s")
print("dict :", 
      f"{timeit(functools.partial(dict_count_words, words) , number=nb_runs):.3f}s")
print("Counter :", 
      f"{timeit(functools.partial(counter_count_words, words), number=nb_runs):.3f}s")


then, if we want to count words occurrences in a text,using Counter or dicts  on 100000 runs:
default_dict : 0.680s
dict : 0.658s
Counter : 0.319s


Now, it is clear that regex and Counter are the fastest way to count words, on top of freeing code space,

Let's see what functionalities offers Counter, on top of being fast :
https://docs.python.org/3/library/collections.html#collections.Counter

In [3]:
# COUNTER POSSIBILITIES
counter = Counter(words)

print("elements of counter :\n", counter.elements())
print("\ntop 4 used words :\n", counter.most_common(4))

print("\nIf you want to compare two texts and showcase"
      "similarities or differences :")
second_text = "a string that have some words to substract from the other text."
second_words = regex_get_words(second_text)
second_counter = counter_count_words(second_words)
counter.subtract(second_words)
print("difference between texts :\n", counter)

print("\nIf you only want to take elements that are on a positive count :\n", +counter)

print("total words :\n", counter.total())


elements of counter :

top 4 used words :
 [('a', 2), ('code', 2), ("Python's", 1), ('fun', 1)]

If you want to compare two texts and showcasesimilarities or differences :
difference between texts :
 Counter({'code': 2, "Python's": 1, 'fun': 1, 'yèt': 1, 'complex': 1, 'Using': 1, 'libraries': 1, 'is': 1, 'a': 1, 'way': 1, 'eâse': 1, 'yöur': 1, 'life': 1, 'arouÑd': 1, 'problems': 1, 'Built-ins': 1, "'fonctionalités'": 1, 'of': 1, 'python': 1, 'like': 1, 're': 1, 'collectioNs': 1, 'or': 1, 'functools': 1, 'are': 1, 'pillars': 1, 'for': 1, 'clean': 1, 'optimized': 1, 'Do': 1, 'not': 1, 'rewrite': 1, 'enhance': 1, 'it': 1, 'Now': 1, 'Dø': 1, 'It': 1, 'Yourself': 1, 'to': 0, 'some': 0, 'string': -1, 'that': -1, 'have': -1, 'words': -1, 'substract': -1, 'from': -1, 'the': -1, 'other': -1, 'text': -1})

If you only want to take elements that are on a positive count :
 Counter({'code': 2, "Python's": 1, 'fun': 1, 'yèt': 1, 'complex': 1, 'Using': 1, 'libraries': 1, 'is': 1, 'a': 1, 'way': 1, 'e

AttributeError: 'Counter' object has no attribute 'total'

Counter.total() should be a thing, according to the doc... But we use a lib version that does not have it.

Also, while we are at it, remember this ? `<itertools.chain object at 0x00....>` we could try and make it so print(counter.elements()) doesn't requires us to iter through with `[elem for elem in counter.elements()]`

Now, if we were to create it ourselves, we could do something like __this__:

In [4]:
# ADDING FUNCTIONALITIES TO COUNTER
class MyCounter(Counter):
    def __init__(self, *args, **kwds):
        """Call to super().__init__(...) creates an instance of Counter"""
        super().__init__(*args, **kwds)

    def total(self):
        """add each word's count"""
        total = 0
        for word, count in self.items():
            total += count
        return total
    
    def elements(self):
        """
        alter counter.elements() to return a list of elements,
        instead of an itertools.chain object
        """
        return [elem for elem in super().elements()]


counter = MyCounter(words)
print("total number of words : ", counter.total())
print("words :\n", counter.elements())

total number of words :  42
words :
 ["Python's", 'fun', 'yèt', 'complex', 'Using', 'libraries', 'is', 'a', 'a', 'way', 'to', 'eâse', 'yöur', 'life', 'arouÑd', 'some', 'problems', 'Built-ins', "'fonctionalités'", 'of', 'python', 'like', 're', 'collectioNs', 'or', 'functools', 'are', 'pillars', 'for', 'clean', 'optimized', 'code', 'code', 'Do', 'not', 'rewrite', 'enhance', 'it', 'Now', 'Dø', 'It', 'Yourself']


### We made it !
You now have a MyCounter class that have a .total() method and .elements() now returns a list.

What else could we do with it ??
